In [ ]:
# This code solves of time-dependent one-dimensional Schrödinger Equation
# with finite difference Crank-Nicolson method for Gaussian wave packet.
# The real and imaginary parts of the wave function are shown on a complex plane.

using GLMakie
using LinearAlgebra
using SpecialFunctions

function AB(V)
    """
        Create arrays for the Crank-Nicolson method
    """
    ones_arr = ones(ComplexF64,stepX-1)
    d = im*λ.-Δx^2*V.-2
    A = Tridiagonal(ones_arr,d,ones_arr)
    d = im*λ.+Δx^2*V.+2
    B = Tridiagonal(-ones_arr,d,-ones_arr)
    A,B
end

# define constants
k₀ = 100*π # proportinal to the moment of wave
σ₀ = 0.05 # wave's width
x₀ = 0.8 # wave's middle point
stepX = 10000 # x-axis grid density
Δt = 1.0e-6 # time step
L = 2.0 # length of the simulation box
Δx = L/stepX # x-axis step
λ = 2*Δx^2/Δt # intermediate constant
x = range(0,L,stepX) |> collect # x-axis array

file_path = "C:/Users/animation.mp4" # choose path to save animation

"done"

In [ ]:
# Test calculation for the initial state
Ψ₀ = exp.(im*k₀*x).*exp.(-(x.-x₀).^2/2/σ₀^2) # Gaussian wave-packet
ρ₀ = abs2.(Ψ₀) # probability distribution function
f1 = Figure()
ax = Axis(f1[1,1])
lines!(x,ρ₀)
f1

In [ ]:
# Define potentials

V = zeros(stepX)
######## Borders to reflect wave function from edges
V[1:500] = [5e6*exp(-200xᵢ) for xᵢ ∈ x[1:500]]
V[9500:end] = [5e6*exp(200(xᵢ-2.0)) for xᵢ ∈ x[9500:end]]
########

# different types of the potential

######## single wall/well
# V[4400:5600] +.= -k₀^2 # box
########

######## double wall/well
# V[3750-600:3750+600] .-= k₀^2
# V[6250-600:6250+600] .-= k₀^2
########

######## harmonic oscillator
V .+= [2.5e5*(xᵢ-1.0)^2 for xᵢ ∈ x]
########

######## "delta" function
# V[5000] .+= 1e6
########

######## gravitation-like
# V .+= [7e4*xᵢ for xᵢ ∈ x]
########


######## Bessel function
# V .+= 1.5e5*besselj.(1.0,15x)
########

draw_V = V/k₀^2 # for depicting potential curve on the graph

A,B = AB(V) # create arrays for the Crank-Nicolson method
Ψ₀ = exp.(im*k₀*x).*exp.(-(x.-x₀).^2/2/σ₀^2) # initialize wave function

# setup the graph, real part and imaginary parts of wave function plotted on Y and Z axes, respectively
fig, ax, l1 = lines(x, real(Ψ₀), imag(Ψ₀), linewidth = 3; figure = (; size = (1920, 1080)),
                    axis = (type = Axis3, xlabel="Distance", ylabel="Re(Ψ)", zlabel="Im(Ψ)",
                            xlabelsize = 28, ylabelsize = 28, zlabelsize = 28,
                            aspect = (3,1,1), azimuth = 245*π/180, elevation=15*π/180, protrusions=(70,10,5,5),
                            limits = (nothing, nothing, -1.1, 1.1, -1.1, 1.1)))
l2 = lines!(ax,x,ones(stepX),abs2.(Ψ₀)) # plot probability density funtion
l3 = lines!(ax,x,ones(stepX),draw_V) # plot potential curve
axislegend(ax, [l1, l2, l3], ["Ψ", "|Ψ|²", "V(x), a.u."], framevisible = false, position = :lt, labelsize = 28)
record(fig, file_path, range(0,1000); framerate = 60) do i # iterates over all frames, and save animation
    Ψ = A\(B*Ψ₀) # solve the Crank-Nicolson method
    global Ψ₀ = Ψ
    l1[2] = real(Ψ₀)
    l1[3] = imag(Ψ₀)
    l2[3] = abs2.(Ψ₀) # update probability density funtion
end